<a href="https://colab.research.google.com/github/hibatallahk/My-Federated-Learning-implementations/blob/main/First_FL_PySyft_2_clients_MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Federated Learning on MNIST using a CNN with PyTorch & PySyft


###related reference: 
PyTorch(https://github.com/pytorch/examples/blob/master/mnist/main.py)

PySyft (https://github.com/OpenMined/PySyft/)

- Credits goes to Archit Garg for the tutorial in school of AI/UDACITY

- Install syft package by pip.

In [ ]:
!pip3 install syft==0.2.9

### Imports and model specifications

In [ ]:
!pip3 install torchvision

     |████████████████████████████████| 750.6 MB 9.2 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0
    Uninstalling torch-1.10.0:
      Successfully uninstalled torch-1.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
syft 0.6.0 requires torch<=1.10.0,>=1.8.1, but you have torch 1.11.0 which is incompatible.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

- Define remote workers `i_1` and `i_2`.

In [ ]:
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch, extra functionalities to support Federated Learning
i_1 = sy.VirtualWorker(hook, id="i_1")  # define remote worker i_1
i_2 = sy.VirtualWorker(hook, id="i_2")  # define remote worker i_2

We define the setting of the learning task

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

### Data loading and sending to workers
- First loading the data and transforming the training Dataset into a Federated Dataset split across the workers using the `.federate` method. 
- This federated dataset is now given to a Federated DataLoader. The test dataset remains unchanged.

In [ ]:
federated_train_loader = sy.FederatedDataLoader( # this is a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((i_1, i_2)), # we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


### CNN specification
- We use a standard CNN

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Define the train and test functions
For the train function, because the data batches are distributed across `alice` and `bob`, you need to send the model to the right location for each batch. Then, you perform all the operations remotely with the same syntax like you're doing local PyTorch. When you're done, you get back the model updated and the loss to look for improvement.

In [ ]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # now it is a distributed dataset
        model.send(data.location) # send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

The test function

In [ ]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### Launch the training

In [ ]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")



Train Epoch: 1 [0/60032 (0%)]	Loss: 2.305134
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.156802
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.896587
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.440329
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.867023
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.654317
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.593097
Train Epoch: 1 [13440/60032 (22%)]	Loss: 0.455294
Train Epoch: 1 [15360/60032 (26%)]	Loss: 0.371373
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.304508
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.314196
Train Epoch: 1 [21120/60032 (35%)]	Loss: 0.369179
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.237825
Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.187933
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.523393
Train Epoch: 1 [28800/60032 (48%)]	Loss: 0.224815
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.143398
Train Epoch: 1 [32640/60032 (54%)]	Loss: 0.267608
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.186926
Train Epoch: 1 [36480/60032 (61%)]	Loss: 0.302966
Train Epoch:

- We trained a model on remote data on 2 clients (`i_1` and `i_2`)using Federated Learning